In [36]:
from nba_api.stats.endpoints import playercareerstats
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.library.parameters import SeasonTypePlayoffs

import json


In [37]:
def json_to_file(f_name, sometext):
    json_object = json.dumps(sometext, indent=4)
    with open(f'data/{f_name}.json', "w") as outfile:
        outfile.write(json_object)


In [ ]:

# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id="203076")
# career.get_data_frames()[0]
career.get_data_frames()[0]

In [ ]:
nba_teams = teams.get_teams()

json_to_file("nba_teams",nba_teams)

In [ ]:
team_abb_collection = []
for i in teams.get_teams():
    team_abb_collection.append(i['abbreviation'])

# team_abb_collection

In [ ]:
# export player to file
all_player_list = players.get_players()
json_to_file("all_player",all_player_list)

In [ ]:
# Create Map for player and playerID

player_id_map = {}
for i in all_player_list:
    player_id_map[i['id']] = i['full_name']
    # break
# player_id_map

In [ ]:
player_list = []

In [ ]:
# Search for player with name
player_name = "Jalen"
player_result = []
for i in all_player_list:
    if i['first_name'] == player_name:
        player_result.append(i)
    elif i['last_name'] == player_name:
        player_result.append(i)

for i in player_result:
    player_list.append(i['id'])



In [ ]:
# Get player career stats

# Check for duplication
player_list = list(set(player_list))

total_df = []
for i in player_list:
    player_career = playercareerstats.PlayerCareerStats(player_id =  str(i)) 
    career_df = player_career.get_data_frames()[0]
    total_df.append(career_df)

combineCareer = pd.concat(total_df)

combineCareer

In [ ]:
# Add player name to the stat and export it to a csv file
combineCareer['PLAYER_NAME'] = combineCareer['PLAYER_ID'].map(player_id_map)

combineCareer_col = combineCareer.columns.tolist()

if 'PLAYER_NAME' not in combineCareer_col[:-1]:
    combineCareer_col = combineCareer_col[-1:] + combineCareer_col[:-1]

combineCareer=combineCareer[combineCareer_col]
combineCareer.to_csv('data/palyer_Stat.csv', sep=',',  index=False)

combineCareer


In [ ]:
# gamefinder = leaguegamefinder.LeagueGameFinder(
#     team_id_nullable=team_select['id'],
#     # team_id_nullable="1610612743",
#     season_nullable="20-21",
#                             # season="20-21",
                            # season_type_nullable=SeasonType.regular
                            # season_type_nullable=SeasonTypePlayoffs.playoffs
                                            #   )  

# games_dict = gamefinder.get_normalized_dict()
# print(SeasonTypePlayoffs.playoffs)
# len(games_dict['LeagueGameFinderResults'])


In [ ]:
match_up_team = "BOS"
matchup_result = []
for i in games_dict['LeagueGameFinderResults']:
    if match_up_team in i['MATCHUP']:
    # if match_up_team.MATCH.str.contsins('MIN:
        matchup_result.append(i)

json_to_file("match_result",matchup_result)

        

In [ ]:
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

# print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')
print(game)

In [ ]:
# Get Playbyplay
from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2('0022301196')
pbp = pbp.get_data_frames()[0]
# pbp.head()

In [ ]:
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022301196') 

# box.away_team_player_stats.get_dict()  

# Data Sets
# box.game.get_dict()                  #equal to box.get_dict()['game']
#box.arena.get_dict()                  #equal to box.get_dict()['game']['arena']
#box.away_team.get_dict()              #equal to box.get_dict()['game']['awayTeam']
#box.away_team_player_stats.get_dict() #equal to box.get_dict()['game']['awayTeam']['players']
#box.away_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.home_team.get_dict()              #equal to box.get_dict()['game']['homeTeam']
#box.home_team_player_stats.get_dict() #equal to box.get_dict()['game']['homeTeam']['players']
#box.home_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.game_details.get_dict()           #equal to box.get_dict()['game'] scrubbed of all other dictionaries
#box.officials.get_dict()              #equal to box.get_dict()['game']['officials']

In [ ]:
# Select the dictionary for the Team (BOS, LAL, IND), which contains their team ID 
team_abb = 'DAL'
team_select = [team for team in nba_teams if team['abbreviation'] == team_abb][0]
team_select
# f'team_id: {team_select["id"]}'


In [ ]:
from datetime import date

gamefinder = leaguegamefinder.LeagueGameFinder(
    # team_id_nullable=team_select['id'],
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular
                            ,date_from_nullable='2025-03-24'
                            # ,date_from_nullable=date.today()
                            # ,date_to_nullable=date.today()
                            # season_type_nullable=SeasonTypePlayoffs.playoffs
                                              )  

games_dict = gamefinder.get_normalized_dict()
# print(SeasonTypePlayoffs.playoffs)
games_dict


In [ ]:
games_dict

In [ ]:
# gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable="1610612742")

games = gamefinder.get_data_frames()[0]


total_game = games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count().loc['2015':]
total_game